In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
from tqdm import tqdm

from plotly import express as px
import plotly.graph_objects as go
import plotly.io as pio


## Functions

In [599]:
def rename_uname(df):
    for i , columns in enumerate(df.columns.levels):
        column_new = columns.tolist()
        for j, row in enumerate(column_new):
            row = str(row)
            if 'Unnamed' in row:
                column_new[j] = ""
            if "NO" == row:
                column_new = ""
        df = df.rename(columns=dict(zip(columns.tolist(), column_new)),
            level = i)
    return df

def categorize_genre(value):
  ending = value[-7:] 
  if ending == "Mujeres":
    return "Mujeres"
  elif ending == "Hombres":
    return "Hombres"
  else:
    return "Ambos sexos"

def renta_rename(df):
    df = rename_uname(df).reset_index(drop=True)
    years = list(set([col[0] for col in df.columns][1:]))
    df.columns = [''.join(str(col)).strip() for col in df.columns.values]
    df.columns = [col.replace(' ','_').lower() for col in df.columns.values]
    df.columns = [col.replace(',','') for col in df.columns.values]


    df = df.rename(columns={
        "('distrito_/_barrio'_'')": 'distrito_barrio',
        f"({years[1]}_'renta_media_por_persona_')": f'renta_media_por_persona_{years[1]}',
        f"({years[1]}_'renta_media_por_hogar')": f'renta_media_por_hogar_{years[1]}',
        f"({years[0]}_'renta_media_por_persona_')": f'renta_media_por_persona_{years[0]}',
        f"({years[0]}_'renta_media_por_hogar')": f'renta_media_por_hogar_{years[0]}',
    })
    df = df.dropna()
    df['distrito_barrio'].replace(regex=True,inplace=True,to_replace=r'\d+\.',value=r'')
    df['distrito_barrio'] = df['distrito_barrio'].str.strip()
    return df

## Paro

In [600]:
paro_madrid = pd.read_excel('../data/madrid/raw/paro/paro_madrid.xlsx',header=[0,1,2])
districts = pd.read_csv('../data/madrid/cleaned/total_by_district.csv')['district'].tolist()
total_by_negh = pd.read_csv('../data/madrid/cleaned/total_by_neighbourhood.csv')
total_by_district = pd.read_csv('../data/madrid/cleaned/popu_by_district.csv')

paro_madrid.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/madrid/cleaned/total_by_district.csv'

In [601]:
paro_madrid = rename_uname(paro_madrid).reset_index(drop=True)
paro_madrid.columns = [' '.join(col).strip() for col in paro_madrid.columns.values]
paro_madrid.columns = [col.replace(' ','_').lower() for col in paro_madrid.columns.values]
paro_madrid = paro_madrid.drop(paro_madrid.columns[0], axis=1)

In [602]:
paro_madrid = paro_madrid.drop(columns=['ciudad_de_madrid_ciudad_de_madrid_ambos_sexos'])
paro_madrid.columns = ['_'.join(col.split('_')[3:]) for col in paro_madrid.columns.values]
paro_madrid = paro_madrid.drop(columns=['no_consta_ambos_sexos'])
paro_madrid.columns = [re.sub(r'\d+_', '', col) for col in paro_madrid.columns.values]
paro_madrid.columns = [re.sub(r'\vallecas', '', col) for col in paro_madrid.columns.values]

In [603]:
total_paro = pd.DataFrame(paro_madrid.sum())
total_paro['district'] = total_paro.index
total_paro.reset_index(drop=True,inplace=True)
total_paro.rename(columns={0:'total'}, inplace=True)
total_paro = total_paro.reindex(columns=['district','total'])
total_paro['total'] = total_paro['total'].astype(int)
total_paro['district'] = total_paro.district.str.title()
total_paro['genre'] = total_paro['district'].apply(categorize_genre)

genre_map = {"_Mujeres": "", "_Hombres": "", "_Ambos_Sexos": ""}
total_paro['district'] = total_paro['district'].replace(genre_map, regex=True)
total_paro['district'] = total_paro['district'].str.replace('_',' ')
total_paro['district'] = total_paro['district'].str.replace('Vallecas ','')

total_paro = total_paro[~total_paro['district'].isin(['No Consta','Ciudad De Madrid'])]
total_paro.to_csv('../data/madrid/cleaned/paro_by_neighbourhood.csv')

In [604]:
total_paro.sort_values(by='total', ascending=False)

,district,total,genre
92,Puente De Vallecas,188449,Ambos sexos
76,Carabanchel,168234,Ambos sexos
68,Latina,138069,Ambos sexos
125,Villaverde,115411,Ambos sexos
107,Ciudad Lineal,108214,Ambos sexos
...,...,...,...
460,Atalaya,277,Mujeres
232,El Plantío,265,Hombres
174,Atocha,265,Hombres
286,Atalaya,179,Hombres


In [605]:
districts = [district.title() for district in districts]

paro_by_neighbour = total_paro[~total_paro['district'].isin(districts)].reset_index(drop=True)
total_by_negh = total_by_negh[~total_by_negh['district'].isin(districts)].reset_index(drop=True)
total_by_negh.to_csv('../data/madrid/cleaned/popu_by_neighbourhood.csv',index=False)
total_by_district.to_csv('../data/madrid/cleaned/popu_by_district.csv', index=False)
paro_by_neighbour.to_csv('../data/madrid/cleaned/paro_by_neighbourhood.csv', index=False)

NameError: name 'districts' is not defined

In [608]:
total_by_district = pd.read_csv('../data/madrid/cleaned/popu_by_district.csv')
distr = total_by_district.district.str.title().tolist()

total_paro_c = total_paro.copy()
districts_good = total_paro_c[total_paro_c.district.isin(distr)].reset_index(drop=True)
districts_good.district = districts_good.district.str.upper()
districts_good.to_csv('../data/madrid/cleaned/paro_by_district.csv')

In [609]:
total_paro_c = total_paro.copy()
total_paro_c[total_paro_c.district.str.contains('Fuencarral')]

,district,total,genre
50,Fuencarral-El Pardo,91537,Ambos sexos
215,Fuencarral-El Pardo,37890,Hombres
389,Fuencarral-El Pardo,53647,Mujeres


## Poblacion

In [610]:
all_madrid = pd.read_excel('../data/madrid/cleaned/population_total.xlsx',header=[1,2])
all_madrid.head()

Acceso a \nBanco Datos  \
                  Índice   
0                  Datos   
1                    NaN   
2                    NaN   
3                    NaN   
4                    NaN   

  Explotación estadística del Padrón Municipal de Habitantes revisado a 1 de enero de 2023  \
                                                                        Unnamed: 1_level_1   
0  1. Población clasificada por Distrito y Sexo, ...                                         
1                                               Edad                                         
2                                                NaN                                         
3                                                NaN                                         
4                                              Total                                         

                                                                               \
  Unnamed: 2_level_1 Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1   
0                NaN                NaN                NaN                NaN   
1   Ciudad de Madrid                NaN                NaN                NaN   
2              Total            Hombres            Mujeres                NaN   
3                NaN                NaN                NaN                NaN   
4            3339931            1559866            1780065                NaN   

                                                                               \
  Unnamed: 6_level_1 Unnamed: 7_level_1 Unnamed: 8_level_1 Unnamed: 9_level_1   
0                NaN                NaN                NaN                NaN   
1         01. Centro                NaN                NaN                NaN   
2              Total            Hombres            Mujeres                NaN   
3                NaN                NaN                NaN                NaN   
4             139687              70770              68917                NaN   

   ... Si desea participar en nuestra encuesta de satisfacción, pinche aquí  \
   ...                                                  Unnamed: 79_level_1   
0  ...                                                NaN                     
1  ...                                                NaN                     
2  ...                                            Hombres                     
3  ...                                                NaN                     
4  ...                                              40580                     

                                                                    \
  Unnamed: 80_level_1 Unnamed: 81_level_1      Unnamed: 82_level_1   
0                 NaN                 NaN                      NaN   
1                 NaN                 NaN  20. San Blas-Canillejas   
2             Mujeres                 NaN                    Total   
3                 NaN                 NaN                      NaN   
4               43224                 NaN                   161219   

                                                               \
  Unnamed: 83_level_1 Unnamed: 84_level_1 Unnamed: 85_level_1   
0                 NaN                 NaN                 NaN   
1                 NaN                 NaN                 NaN   
2             Hombres             Mujeres                 NaN   
3                 NaN                 NaN                 NaN   
4               76154               85065                 NaN   

                                                               
  Unnamed: 86_level_1 Unnamed: 87_level_1 Unnamed: 88_level_1  
0                 NaN                 NaN                 NaN  
1         21. Barajas                 NaN                 NaN  
2               Total             Hombres             Mujeres  
3                 NaN                 NaN                 NaN  
4               48646               23332               25314  

[5 rows x 89 columns]

In [611]:
total = all_madrid[all_madrid.columns[all_madrid.columns.str.contains('total')]].reset_index(drop=True)
total.columns = [col.replace('total','') for col in total.columns.values]
total.columns = ['_'.join(col.split('_')[1:]) for col in total.columns.values]

AttributeError: Can only use .str accessor with Index, not MultiIndex

In [612]:
total.columns

NameError: name 'total' is not defined

In [ ]:
total_values = pd.DataFrame(total.sum())
total_values['district'] = total_values.index
total_values.reset_index(drop=True,inplace=True)
total_values.rename(columns={0:'total'}, inplace=True)
total_values = total_values.reindex(columns=['district','total'])
#total_values['total'] = total_values['total'].astype(int)
total_values['district'] = total_values.district.str.upper()
total_values['district'] = total_values['district'].str.replace('_','')
total_values['district'] = total_values['district'].str.replace('-',' - ')

total_values.to_csv('../data/madrid/total_by_district.csv')

## Vivienda

In [640]:

df_housing = pd.read_excel('../data/madrid/raw/vivienda_segunda_mano.xlsx')
df_housing = df_housing.replace('-',0)
df_housing.rename(columns={'Unnamed: 0':'district', 'Unnamed: 1':'barrio'}, inplace=True)
df_housing['district'].replace(regex=True,inplace=True,to_replace=r'\d+\.',value=r'')
df_housing['barrio'].replace(regex=True,inplace=True,to_replace=r'\d+\.',value=r'')
_
df_housing_yearly = pd.melt(df_housing, id_vars=['barrio', 'district'], var_name='year', value_name='price')
df_housing_yearly['price'] = df_housing_yearly['price'].astype(float)

df_housing_yearly.head()

/tmp/ipykernel_134692/932187357.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/tmp/ipykernel_134692/932187357.py:5: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' 

,barrio,district,year,price
0,Ciudad de Madrid,Ciudad de Madrid,2017,3.285
1,Centro,Centro,2017,4.599
2,Palacio,Centro,2017,4.474
3,Embajadores,Centro,2017,4.017
4,Cortes,Centro,2017,4.842


In [641]:
price_district_m2 = df_housing_yearly.groupby(['district','year']).agg({'price':'sum'}).reset_index()
price_neighbourhood_m2 = df_housing_yearly.groupby(['barrio','year']).agg({'price':'sum'}).reset_index()
price_neighbourhood_m2

,barrio,year,price
0,Abrantes,2017,1.644
1,Abrantes,2018,1.896
2,Abrantes,2019,1.883
3,Abrantes,2020,1.925
4,Abrantes,2021,1.897
...,...,...,...
961,Ciudad de Madrid,2018,3.844
962,Ciudad de Madrid,2019,3.732
963,Ciudad de Madrid,2020,3.682
964,Ciudad de Madrid,2021,3.700


In [642]:

df_housing_month = pd.read_excel('../data/madrid/raw/vivienda_segunda_mano_mes.xlsx')
df_housing_month = df_housing_month.replace('-',0)
df_housing_month.rename(columns={'Unnamed: 0':'año', 'Unnamed: 1':'district'}, inplace=True)
df_housing_month['district'].replace(regex=True,inplace=True,to_replace=r'\d+\.',value=r'')
df_housing_month['district'] = df_housing_month['district'].str.strip()

df_housing_month = pd.melt(df_housing_month, id_vars=['año', 'district'], var_name='month', value_name='price')
df_housing_month['price'] = df_housing_month['price'].astype(float)

df_housing_month

/tmp/ipykernel_134692/4075314226.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,año,district,month,price
0,2006,Ciudad de Madrid,Enero,0.000
1,2006,Centro,Enero,0.000
2,2006,Arganzuela,Enero,0.000
3,2006,Retiro,Enero,0.000
4,2006,Salamanca,Enero,0.000
...,...,...,...,...
4759,2023,Villa de Vallecas,Diciembre,2.691
4760,2023,Vicálvaro,Diciembre,2.846
4761,2023,San Blas-Canillejas,Diciembre,2.852
4762,2023,Barajas,Diciembre,3.406


In [643]:
month_map = {'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
             'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12}

In [644]:
import pandas as pd
centro = df_housing_month[df_housing_month['district'] == 'Centro'].reset_index(drop=True)
#convert month_n into month numbers
centro['month'] = centro['month'].map(month_map).astype(int)
centro['date'] = pd.to_datetime(centro['año'].astype(str) + '-' + centro['month'].astype(str))
centro.index = centro['date']
centro['price'].pct_change()

date
2006-01-01         NaN
2007-01-01         NaN
2008-01-01         inf
2009-01-01   -0.047543
2010-01-01   -0.022318
                ...   
2019-12-01    0.017293
2020-12-01   -0.038347
2021-12-01   -0.004317
2022-12-01    0.074525
2023-12-01    0.129107
Name: price, Length: 216, dtype: float64

In [645]:
# Resample to quarterly frequency, taking the last month of each quarter
quarterly = centro['price'].resample('YE').mean()
quarterly.pct_change

<bound method NDFrame.pct_change of date
2006-12-31    0.000000
2007-12-31    2.623583
2008-12-31    4.293167
2009-12-31    4.093417
2010-12-31    3.999667
2011-12-31    3.881833
2012-12-31    3.571167
2013-12-31    3.289833
2014-12-31    3.278000
2015-12-31    3.423833
2016-12-31    3.666333
2017-12-31    4.135583
2018-12-31    4.874083
2019-12-31    5.024000
2020-12-31    4.994833
2021-12-31    4.729250
2022-12-31    5.125583
2023-12-31    2.785083
Freq: YE-DEC, Name: price, dtype: float64>

In [646]:
fig = px.line(df_housing_month,
              x='month',
              y='price',
              animation_frame='district',
              title='Evolution of price/m2 by district',
              color='año')

pio.write_html(fig, file='../data/html/evolution_by_month.html')

In [647]:
df_rent = pd.read_excel('../data/madrid/raw/renta.xlsx')
df_rent.head()

,Atlas de distribución de renta de los hogares,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,Indicadores de renta media. 2016-2017,NaN,NaN,NaN,NaN
2,Unidades: Euros,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,Distrito / Barrio,2017,NaN,2016,NaN


## Renta

In [648]:
renta_15_16 = pd.read_excel('../data/madrid/raw/renta/renta_15_16.xlsx',skiprows=[0,1,2,3,4], header=[0,1])  
renta_16_17 = pd.read_excel('../data/madrid/raw/renta/renta17_16.xlsx', skiprows=[0,1,2,3,4], header=[0,1])    
renta_17_18 = pd.read_excel('../data/madrid/raw/renta/renta17_18.xlsx', skiprows=[0,1,2,3,4], header=[0,1])    
renta_18_20 = pd.read_excel('../data/madrid/raw/renta/renta19_20.xlsm', skiprows=[0,1,2,3,4], header=[0,1])   

In [622]:
renta_15_16 = renta_rename(renta_15_16)
renta_16_17 = renta_rename(renta_16_17)
renta_17_18 =  renta_rename(renta_17_18)
renta_19_20 = renta_rename(renta_18_20)

/tmp/ipykernel_134692/3967477780.py:39: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/tmp/ipykernel_134692/3967477780.py:39: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=Tru

## Points

In [623]:
points_raw = pd.read_csv('../data/points/amenity.csv')
points_raw.shape

(32672, 10)

## Dataset Creation

In [624]:
#need to find which categories we wanna locate 
df_amenity = pd.read_csv('../data/points/amenity.csv')
df_amenity.category.unique()

array(['fuel', nan, 'cinema', 'parking_entrance', 'restaurant', 'pub',
       'theatre', 'police', 'parking', 'place_of_worship', 'fast_food',
       'pharmacy', 'bar', 'retail', 'taxi', 'bicycle_parking', 'cafe',
       'bank', 'bench', 'university', 'clinic', 'drinking_water',
       'post_box', 'post_office', 'recycling', 'veterinary', 'atm',
       'social_facility', 'dentist', 'kindergarten', 'nightclub',
       'bicycle_rental', 'library', 'waste_disposal', 'ice_cream',
       'internet_cafe', 'fountain', 'school', 'music_school',
       'language_school', 'dancing_school', 'vending_machine',
       'marketplace', 'bus_station', 'toilets', 'telephone',
       'arts_centre', 'college', 'training', 'hospital', 'doctors',
       'car_rental', 'casino', 'water_point', 'waste_basket',
       'social_centre', 'courthouse', 'car_wash', 'biergarten',
       'community_centre', 'driving_school', 'vacuum_cleaner', 'prison',
       'brothel', 'satellite_dish', 'grave_yard', 'food_court',
  

In [625]:
df_housing = pd.read_csv('../data/madrid/cleaned/fotocasa/fotocasa_2023_located_districts.csv')
df_housing.shape

(14130, 19)

In [626]:
df_housing.rename(columns={'district_geolocated':'distrito_barrio'}, inplace=True)

In [627]:
df_housing.columns = df_housing.columns.str.lower()
df_housing['zipcode'] = df_housing['zipcode'].fillna(0)
df_housing['zipcode'] = df_housing['zipcode'].astype(int)
df_housing['house_id'] = df_housing.index

In [628]:
df_airbnb = pd.read_csv('../data/points/points_count/housing_airbnb.csv')
df_college = pd.read_csv('../data/points/points_count/housing_college.csv')
df_dentist = pd.read_csv('../data/points/points_count/housing_dentist.csv')
df_fast_food = pd.read_csv('../data/points/points_count/housing_fast_food.csv')
df_jewelry = pd.read_csv('../data/points/points_count/housing_jewelry.csv')
df_university = pd.read_csv('../data/points/points_count/housing_university.csv')
df_restaurant = pd.read_csv('../data/points/points_count/housing_restaurant.csv')
df_hospital = pd.read_csv('../data/points/points_count/housing_hospital.csv')
df_pharmacy = pd.read_csv('../data/points/points_count/housing_pharmacies.csv')
df_supermarket = pd.read_csv('../data/points/points_count/housing_supermarket.csv')
df_airbnb.shape, df_college.shape, df_dentist.shape, df_fast_food.shape, df_jewelry.shape, df_university.shape, df_restaurant.shape, df_hospital.shape

((3237335, 2),
 (6265, 2),
 (28298, 2),
 (105914, 2),
 (19743, 2),
 (1312, 2),
 (682458, 2),
 (955, 2))

In [629]:
df_airbn = df_airbnb.groupby('house_id').agg(count=(df_airbnb.columns[1],'count')).reset_index()
df_college = df_college.groupby('house_id').agg(count=(df_college.columns[1],'count')).reset_index()
df_dentist = df_dentist.groupby('house_id').agg(count=(df_dentist.columns[1],'count')).reset_index()
df_fast_food = df_fast_food.groupby('house_id').agg(count=(df_fast_food.columns[1],'count')).reset_index()
df_jewelry = df_jewelry.groupby('house_id').agg(count=(df_jewelry.columns[1],'count')).reset_index()
df_university = df_university.groupby('house_id').agg(count=(df_university.columns[1],'count')).reset_index()
df_restaurant = df_restaurant.groupby('house_id').agg(count=(df_restaurant.columns[1],'count')).reset_index()
df_hospital = df_hospital.groupby('house_id').agg(count=(df_hospital.columns[1],'count')).reset_index()
df_pharmacy = df_pharmacy.groupby('house_id').agg(count=(df_pharmacy.columns[1],'count')).reset_index()
df_supermarket = df_supermarket.groupby('house_id').agg(count=(df_supermarket.columns[1],'count')).reset_index()

In [630]:
df_housing_left = df_housing.merge(df_airbn, on='house_id', how='left', suffixes=('', '_airbnb'))
df_housing_left = df_housing_left.merge(df_college, on='house_id', how='left', suffixes=('', '_college_500'))
df_housing_left = df_housing_left.merge(df_dentist, on='house_id', how='left', suffixes=('', '_dentist_500'))
df_housing_left = df_housing_left.merge(df_fast_food, on='house_id', how='left', suffixes=('', '_fast_food_500'))
df_housing_left = df_housing_left.merge(df_jewelry, on='house_id', how='left', suffixes=('', '_jewelry_500'))
df_housing_left = df_housing_left.merge(df_university, on='house_id', how='left', suffixes=('', '_university_500'))
df_housing_left = df_housing_left.merge(df_restaurant, on='house_id', how='left', suffixes=('', '_restaurant_500'))
df_housing_left = df_housing_left.merge(df_hospital, on='house_id', how='left', suffixes=('', '_hospital_500'))
df_housing_left = df_housing_left.merge(df_pharmacy, on='house_id', how='left', suffixes=('', '_pharmacy_500'))
df_housing_left = df_housing_left.merge(df_pharmacy, on='house_id', how='left', suffixes=('', '_supermarket_500'))


In [631]:
df_housing_left = df_housing_left.rename(columns={'count':'count_airbnb_500'})

In [632]:
df_housing_left[df_housing_left.columns[df_housing_left.columns.str.contains('count')]].fillna(0, inplace=True)

/tmp/ipykernel_134692/2548365896.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [633]:
df_housing_left = df_housing_left[~df_housing_left['neighbourhood'].isna()]
df_housing_left = df_housing_left.fillna(0)
df_housing_left.isna().sum()

address                  0
zipcode                  0
longitude                0
latitude                 0
price                    0
date                     0
rooms                    0
bathrooms                0
surface                  0
floor                    0
elevator                 0
air_conditioner          0
heater                   0
parking                  0
balcony                  0
terrace                  0
swimming_pool            0
neighbourhood            0
distrito_barrio          0
house_id                 0
count_airbnb_500         0
count_college_500        0
count_dentist_500        0
count_fast_food_500      0
count_jewelry_500        0
count_university_500     0
count_restaurant_500     0
count_hospital_500       0
count_pharmacy_500       0
count_supermarket_500    0
dtype: int64

In [634]:
df_housing_left2 = pd.merge(df_housing_left, renta_15_16, on='distrito_barrio', how='inner')
df_housing_left2 = pd.merge(df_housing_left2, renta_16_17, on='distrito_barrio', how='inner')
df_housing_left2 = pd.merge(df_housing_left2, renta_17_18, on='distrito_barrio', how='inner')
df_housing_left2 = pd.merge(df_housing_left2, renta_19_20, on='distrito_barrio', how='inner')
df_housing_left2

,address,zipcode,longitude,latitude,price,date,rooms,bathrooms,surface,floor,...,renta_media_por_persona_2016_y,renta_media_por_hogar_2016_y,renta_media_por_persona_2018,renta_media_por_hogar_2018,renta_media_por_persona_2017_y,renta_media_por_hogar_2017_y,renta_media_por_persona_2020,renta_media_por_hogar_2020,renta_media_por_persona_2019,renta_media_por_hogar_2019
0,Retiro,28007,40.402584,-3.672911,445000,2023-06-07,2,2,102,0,...,21504.0,52963.0,22706.0,55775.0,21598.0,53111.0,23227.0,56302.0,23262.0,57027.0
1,Calle Amparo Centro,28012,40.407387,-3.701162,235000,2023-04-16,2,1,68,0,...,16147.0,32458.0,17932.0,36072.0,16711.0,33473.0,18314.0,36984.0,18789.0,38171.0
2,Barrio de Salamanca,28001,40.420685,-3.688424,2350000,2022-09-09,2,2,177,7,...,24433.0,57147.0,26255.0,61466.0,24683.0,57720.0,25932.0,60401.0,25770.0,60435.0
3,Chamberí,28010,40.436382,-3.698839,1100000,2023-06-12,4,2,167,10,...,22499.0,50882.0,24112.0,54327.0,22897.0,51644.0,24913.0,55757.0,24881.0,56073.0
4,Calle de la Laguna Carabanchel,28025,40.387520,-3.741732,139000,2023-06-15,2,1,70,9,...,10872.0,28301.0,11327.0,29842.0,10988.0,28721.0,11713.0,30890.0,11679.0,31190.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12473,De las Minas Centro,28004,40.425182,-3.706110,658000,2023-06-13,2,2,128,9,...,16147.0,32458.0,17932.0,36072.0,16711.0,33473.0,18314.0,36984.0,18789.0,38171.0
12474,Federico Grases Carabanchel,28025,40.375540,-3.744016,115000,2023-06-08,2,1,63,8,...,10872.0,28301.0,11327.0,29842.0,10988.0,28721.0,11713.0,30890.0,11679.0,31190.0
12475,Avenida de las Águilas Latina,28044,40.382534,-3.772753,164500,2023-05-31,2,1,60,7,...,12232.0,30490.0,12749.0,31872.0,12370.0,30847.0,13261.0,32957.0,13179.0,33237.0
12476,Calle de Pablo Sarasate Latina,28047,40.404716,-3.731530,174900,2023-06-05,3,2,73,0,...,12232.0,30490.0,12749.0,31872.0,12370.0,30847.0,13261.0,32957.0,13179.0,33237.0


In [4]:
paro_by_neighbour = pd.read_csv('../data/madrid/cleaned/paro_by_neighbourhood.csv')
paro_by_district = pd.read_csv('../data/madrid/cleaned/paro_by_district.csv')

In [5]:
paro_by_neighbour = paro_by_neighbour.rename(columns={'district':'neighbourhood','total':'paro_total'})

In [6]:
paro_by_neighbour_women = paro_by_neighbour[paro_by_neighbour['genre'] == 'Mujeres'][['neighbourhood','paro_total']]
paro_by_neighbour_women = paro_by_neighbour_women.rename(columns={'paro_total':'paro_women'})


paro_by_neighbour_men = paro_by_neighbour[paro_by_neighbour['genre'] == 'Hombres'][['neighbourhood','paro_total']]
paro_by_neighbour_men = paro_by_neighbour_women.rename(columns={'paro_total':'paro_men'})


paro_by_neighbour_total = paro_by_neighbour[paro_by_neighbour['genre'] == 'Ambos sexos'][['neighbourhood','paro_total']]
paro_by_neighbour_total = paro_by_neighbour_women.rename(columns={'paro_total':'paro_all'})


In [7]:
df_housing_left2 = pd.merge(df_housing_left2, paro_by_neighbour_total, on='neighbourhood', how='inner')

df_housing_left2.columns

NameError: name 'df_housing_left2' is not defined

In [8]:
suffix = "_y" 
cols_to_drop = df_housing_left2.filter(like=suffix).columns
df_housing_left2 = df_housing_left2.drop(columns=cols_to_drop)

fig = plt.figure(figsize=(12,12))

correlation_matrix = df_housing_left2.select_dtypes(exclude='object').corr()
sns.heatmap(correlation_matrix, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

NameError: name 'df_housing_left2' is not defined

## To remove

In [ ]:
df_housing['point'] = df_housing.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)

In [ ]:
df_housing[['address','point']].head()

In [ ]:
df_restaurants = points_raw[points_raw['category'] == 'restaurant']
df_restaurants.shape

In [ ]:
points_raw[['id','category']].groupby('category').count().reset_index().sort_values(by='id', ascending=False)

In [ ]:
df_count = pd.read_csv('../data/points/housing_school.csv')
df_count.shape

In [ ]:
df_count[df_count['house_id'] == 0]

In [ ]:
df_count[df_count['house_id'] == 1300]

In [ ]:
df_count.house_id.nunique()

In [ ]:
df_count.shop_id.nunique()